<a href="https://colab.research.google.com/github/catchlui/masters_landmark/blob/master/ImagePreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 
import sys, requests, shutil, os,multiprocessing,urllib.request
from tqdm import tqdm
import logging
#logging.basicConfig(filename="/content/gdrive/My Drive/Colab Notebooks/master_research/downloader.log", format="%(asctime)-15s %(levelname)s %(message)s",
                       # datefmt="%F %T", level=logging.DEBUG)

In [0]:
data=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/train.csv')


In [0]:
data = data.iloc[:20000,:]



In [0]:
data.head()

,id,url,landmark_id
0,97c0a12e07ae8dd5,http://lh4.ggpht.com/-f8xYA5l4apw/RSziSQVaABI/...,6347
1,650c989dd3493748,https://lh5.googleusercontent.com/-PUnMrX7oOyA...,12519
2,05e63ca9b2cde1f4,http://mw2.google.com/mw-panoramio/photos/medi...,264
3,08672eddcb2b7c93,http://lh3.ggpht.com/-9fgSxDYwhHA/SMvGEoltKTI/...,13287
4,fc49cb32ef7f1e89,http://lh6.ggpht.com/-UGAXxvPbr98/S-jGZbyMIPI/...,4018


In [0]:
### Let's delete landmarks without IDs

blank_idx = data[data['landmark_id']=='None'].index

In [0]:
data.drop(blank_idx,inplace=True)

In [0]:
data.landmark_id.value_counts()

6051     846
9633     796
6599     408
9779     308
2061     216
        ... 
8555       1
342        1
1521       1
13029      1
12168      1
Name: landmark_id, Length: 5018, dtype: int64

In [0]:
### create a list out of this
landmark_dict_key = data.landmark_id.value_counts().keys().tolist()
landmark_dict_value = data.landmark_id.value_counts().tolist()
landmark_dict = dict(zip(landmark_dict_key,landmark_dict_value))


In [0]:
landmark_dict_final = {key:value for (key,value) in landmark_dict.items() if value >10 }

In [0]:
landmark_dict_landmark_list = list(landmark_dict_final.keys())

In [0]:
new_df = data[data['landmark_id'].isin(landmark_dict_landmark_list)]

In [0]:
len(new_df)

8837

In [0]:

new_df.to_csv ('/content/gdrive/My Drive/Colab Notebooks/master_research/train_8837.csv')

In [0]:
data_frame = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/train_8837.csv')

In [0]:
len(data_frame)

8837

In [0]:
def ParseData(data_file):
    #csvfile = open(data_file, 'r')
    #csvreader = csv.reader(csvfile)
    key_url_list = [line[:3] for line in zip(data_file['id'],data_file['url'],data_file['landmark_id'])]
    return key_url_list  # Chop off header

In [0]:
url_list = ParseData(data_frame)

In [0]:
url_list[5]

('d369520abdc48e11',
 'https://lh6.googleusercontent.com/-xIJ3I2i3ESk/Ud-m9bYtxoI/AAAAAAAAdho/l-zOMG1nosw/s1600/',
 6511)

In [0]:
from PIL import Image
import requests

In [0]:
def ValidateURL(key_url):
    (key, url,landmark_id) = key_url
    #print(key,url,landmark_id)
    
    try:
        img = Image.open(requests.get(url, stream=True).raw)
        #print('its fine ')
        temp.append(key_url)
    except:
        #print('error:in response')
        #logging.warning('Warning: Could not download image %s from %s' % (key, url))
        rejected.append(key_url)
    return 

In [0]:
temp = []
rejected = []
for i in range(len(url_list)):
  if i/1000 == 0:
    print(i)
  ValidateURL(url_list[i])



0


In [0]:
rejected = pd.DataFrame(rejected)

In [0]:
working = pd.DataFrame(temp)

In [0]:
len(temp)

8364

In [0]:
working.rename(columns={0:'id',1:'url',2:'landmark_id'},inplace=True)

In [0]:
working.to_csv ('/content/gdrive/My Drive/Colab Notebooks/master_research/train_working_8364.csv')

In [0]:
working = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/train_working_8364.csv')

In [0]:
working.head()

,Unnamed: 0,id,url,landmark_id
0,0,97c0a12e07ae8dd5,http://lh4.ggpht.com/-f8xYA5l4apw/RSziSQVaABI/...,6347
1,1,650c989dd3493748,https://lh5.googleusercontent.com/-PUnMrX7oOyA...,12519
2,2,056708de792326b9,https://lh3.googleusercontent.com/-nbz_XT1dGz4...,5046
3,3,6ae1e206c579f649,https://lh6.googleusercontent.com/-onL5-pK9nZI...,5554
4,4,7b1e476a7c89bb34,http://lh5.ggpht.com/-3GWjPriaA9g/Sg3e_S0qEgI/...,12220


In [0]:
def DownloadImage(key_url):
    (key, url,landmark_id) = key_url
    #out_dir ='/content/gdrive/My Drive/Colab Notebooks/master_research/test_images_from_train/'
    out_dir ='gdrive/My Drive/Colab Notebooks/master_research/train_images/'
    #out_dir ='/content/gdrive/My Drive/Colab Notebooks/master_research/validation_images_model/'
    
    landmark = str(landmark_id)
    path = out_dir+ landmark
    if not os.path.exists(path):
      os.makedirs(path,exist_ok=True)
    #path=out_dir
    filename = os.path.join(path, '%s.jpg' % key)
    print(filename)

    try:
        #print('in response')
        response=requests.get(url, stream=True)
        #response = urllib.request.urlopen(url)
        with open(filename, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
          #del response
       
    except:
        print('error:in response')
        logging.warning('Warning: Could not download image %s from %s' % (key, url))
        return

# New Section

In [0]:
#key_url_list_test=ParseData(data_test)
#key_url_list_valid=ParseData(data_valid)
key_url_list_train=ParseData(working)

In [0]:
key_url_list_train

In [0]:
def Run():
    #return print('COMMENT ME ! Please run this code on your machine not on a kernel and comment this line :)')
    #print(out_dir)
    #if not os.path.exists(out_dir):
        #os.mkdir(out_dir)
    #print(data_file)
    #key_url_list = ParseData(data_file)
    pool = multiprocessing.Pool(processes=100)  #Adjust number of process regarding to your machine performance
    try:
        with tqdm(total=len(working)) as pbar:
            for i, _ in tqdm(enumerate(pool.imap_unordered(DownloadImage,key_url_list_train))):
                pbar.update()
    except KeyboardInterrupt:
        logging.warning("got Ctrl+C")
    finally:
        pool.terminate()
        pool.join()

In [0]:
Run()

In [0]:
from IPython.display import Image
from IPython.core.display import HTML 

def displayLandmarkImagesLarge(urls, category_name):
    img_style = "width: 200px; height:160px; margin: 0px; float: left; border: 1px solid black;"
    images_list = ''.join([f"<img style='{img_style}' src='{u}' />" for _, u in urls.head(12).iteritems()])
    display(HTML(images_list))

category = data['landmark_id'].value_counts().keys()[15]
urls = data[data['landmark_id'] == category]['url']
displayLandmarkImagesLarge(urls, "")

In [0]:
# Visualize 6 images for each of the first 4 landmarks, ordered by the number of occurences.
LANDMARK_NUMBER = 4
IMAGES_NUMBER = 6
landMarkIDs = pd.Series(data['landmark_id'].value_counts().keys())[0:LANDMARK_NUMBER]
for landMarkID in landMarkIDs:
    url = data[data['landmark_id'] == landMarkID]['url'].head(IMAGES_NUMBER)
    displayLandmarkImagesLarge(url, "")

In [0]:
pd.Series(data['landmark_id'].value_counts().keys())[1:4]

1    6051
2    None
3    6599
dtype: object

In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model = VGG16(weights = 'imagenet', include_top = True)

img_path = r'/content/gdrive/My Drive/Colab Notebooks/master_research/train_images_model/1147/5cefb03a8c8ad42a.jpg'
img = image.load_img(img_path, target_size= (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

preds = model.predict(x)
print ('Predicted:', decode_predictions(preds, top = 3)[0])

Using TensorFlow backend.


40960/35363 [==================================] - 0s 0us/step
Predicted: [('n02859443', 'boathouse', 0.19038849), ('n03028079', 'church', 0.13606061), ('n03933933', 'pier', 0.10720706)]


In [0]:
train_data_dir = '/content/gdrive/My Drive/Colab Notebooks/master_research/train_images_model'
validation_data_dir = '/content/gdrive/My Drive/Colab Notebooks/master_research/validation_images_model'

#counts the number of files in subdirectories of a given directory. 
#outputs the count of images and label of each image in order of reading.

def count(dir):
    i = 1000
    count = []
    while i <1500:
        f = str(i)
#         print (f)
        for root, dirs, files in os.walk(dir +'/'+ f):  # loop through startfolders
            count.append(f)  
        i += 1
    
    return ([len(count),count])

#nb_train_samples = count(train_data_dir)
print('completed')
nb_validation_samples = count(validation_data_dir)

print('completed')
# nb_train/validation_samples is a list of lists of the format [len(count),count]. len(count) is the total number of images in 
# train/validation folder. count is a list of class labels i.e. landmark IDs of the images read.

completed
completed


In [0]:
nb_validation_samples[0]

415

In [0]:
img_width, img_height = 96, 96 # dimensions of downloaded images.
top_model_weights_path = '/content/gdrive/My Drive/Colab Notebooks/master_research/bottleneck_fc_model.h5' # A file with this name would be saved later in the code
epochs = 5
batch_size = 400 #As found in the previous step
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=30,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range = 0.5,
                                 brightness_range = [0.5,1.5])

#     build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(96,96,3))
    print ('start1')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=False) # our data will be in order
    
#     the predict_generator method returns the output of a model, given a generator that yields batches of numpy data
    print ('start2')
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples[0] // batch_size) 
    print(type(bottleneck_features_train))
    print ('bottleneck_features_trained')
    
    
    with open('/content/gdrive/My Drive/Colab Notebooks/master_research/bottleneck_features_train.npy', 'wb') as features_train_file:
        np.save(features_train_file, bottleneck_features_train)
    print ('Train done')
    

## Had to remove (2061 - a32ee99990779b22.jpg) because was throwing an error
    datagen = ImageDataGenerator(rescale=1. / 255) #No image augmentation in validation dataset
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print ('validation predict start')
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples[0] // batch_size)
    #print(bottleneck_features_validation.shape)
    
    with open('/content/gdrive/My Drive/Colab Notebooks/master_research/bottleneck_features_validation.npy', 'wb') as features_validation_file:
        np.save(features_validation_file, bottleneck_features_validation)
    print ('validation done')
save_bottleneck_features()


start1
Found 24507 images belonging to 500 classes.
start2


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


<class 'numpy.ndarray'>
bottleneck_features_trained
Train done
Found 10694 images belonging to 415 classes.
validation predict start


OSError: ignored

In [0]:
epochs = 5
batch_size = 490
import numpy as np

def train_top_model():
    train_data = np.load(open('/content/gdrive/My Drive/Colab Notebooks/master_research/bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(nb_train_samples[1])
    print(type(train_data))
    print('hela')
    print(train_data.shape[1:])
    train_labels = [str(int(train_label) - 1000) for train_label in train_labels] 
    #Had to subtract 1000 because class labels should start from 0. In this case, class labels had a range from 1000 to 2999. 
#     print (train_labels)
    validation_data = np.load(open('/content/gdrive/My Drive/Colab Notebooks/master_research/bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(nb_validation_samples[1])
    validation_labels = [str(int(validation_label) - 1000) for validation_label in validation_labels]
    
#     print (validation_labels)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    n_class = 500 #number of classes fed to the model
    model.add(Dense(n_class, activation='softmax'))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels = to_categorical(train_labels, n_class)
    validation_labels = to_categorical(validation_labels, n_class)
    
    print ('model fit starting')
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

In [0]:
read_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/train_torch.csv')
df_len = len(read_df)


In [0]:
read_df.head()

,Unnamed: 0,id,url,landmark_id
0,0,192333daaf6119cf,https://lh5.googleusercontent.com/-eoC653AtqLs...,3
1,1,f0a4816a71b15ec1,https://lh4.googleusercontent.com/-uu142fuPpro...,4
2,2,c6ec863b149c03c2,http://lh6.ggpht.com/-7Uu23vhvieY/Sehon7kKivI/...,1
3,3,8be00a76fa8840fb,http://lh5.ggpht.com/-Dgd-SG5HB7w/SJYZzEcui-I/...,9
4,4,5b85c8036ee4400d,https://lh5.googleusercontent.com/-fdumkstv_9c...,7


In [0]:
temp = pd.DataFrame(columns=read_df.columns,index=range(0,len(read_df)))

In [0]:
temp.head()

,Unnamed: 0,id,url,landmark_id
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [0]:
 j = 0
for i in range(0,len(read_df)):
 
  dir_path = '/content/gdrive/My Drive/Colab Notebooks/master_research/train_images/'+ read_df.iloc[i]['id']+'.jpg'
  #print(dir_path)
  if os.path.exists(dir_path):
    #print(j)
    try:
      #print('he')
      img = Image.open(dir_path)
      print('here')
      temp.iloc[j] = read_df.iloc[i]
      j = j+1
    except:
      print('error occurred')
     
    
    #print(temp.iloc[j])
    
  else:
    continue
    
  

In [0]:
temp.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
temp.to_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/data_test_torch_down.csv')

In [0]:
hello = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/master_research/data_test_torch_down.csv',index_col=None)

In [0]:
hello.head()

,Unnamed: 0,id,url,landmark_id
0,0,NaN,NaN,NaN
1,1,NaN,NaN,NaN
2,2,NaN,NaN,NaN
3,3,NaN,NaN,NaN
4,4,NaN,NaN,NaN


In [0]:
from PIL import Image
from torch.utils.data.dataset import Dataset
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
class CustomDatasetFromImages(Dataset):
    def __init__(self, csv_path):
        
        self.to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(
                           mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225]
                            )
        self.resize = transforms.Resize(224)
        self.crop = transforms.CenterCrop(224)
        self.data_info = pd.read_csv(csv_path, header=0)
        self.image_arr = np.asarray(self.data_info.iloc[:, 1])
        self.label_arr = np.asarray(self.data_info.iloc[:, 3])
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        single_image_name = self.image_arr[index]+'.jpg'
        single_folder_name = self.label_arr[index]
        dir_path = r'/content/gdrive/My Drive/Colab Notebooks/master_research/train_images/'+str(single_folder_name)+'/'+str(single_image_name)
        

        try:
          img = Image.open(dir_path)
          print('size of the image')
          print(img.size)
          img = self.resize(img)
          img = self.crop(img)
          img_as_tensor = self.to_tensor(img)
          img_as_tensor = self.normalize(img_as_tensor)
          single_image_label = self.label_arr[index]
          return (img_as_tensor,single_image_label)
          #return img.size(
        except:
          print("An exception occurred"+dir_path)
          pass 
     
    def __len__(self):
        return self.data_len

if __name__ == "__main__":
    # Call dataset

    landmarks_train = CustomDatasetFromImages('/content/gdrive/My Drive/Colab Notebooks/master_research/train_working_8364.csv')

In [0]:
hello =landmarks_train.__getitem__(5)

/content/gdrive/My Drive/Colab Notebooks/master_research/train_images/6511/d369520abdc48e11.jpg


In [0]:
train_dataset = datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/master_research/train_images/')

In [0]:
train_dataset.classes

In [0]:
batch_size = 5
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(landmarks_train)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(landmarks_train, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(landmarks_train, batch_size=batch_size, sampler=valid_sampler)

In [0]:
def imshow(axis, inp):
    """Denormalize and show"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    axis.imshow(inp)

In [0]:
training_loader_iter = iter(train_loader)

In [0]:
for idx in range(3):
  img, label = next(training_loader_iter)
  fig = plt.figure(1, figsize=(16, 4))
  grid = ImageGrid(fig, 111, nrows_ncols=(1, 4), axes_pad=0.05)
  for i in range(4):
      ax = grid[i]
      imshow(ax, img[i])  

In [0]:
use_gpu = torch.cuda.is_available()
volatile=True
resnet = models.resnet50(pretrained=True)
inputs, labels = next(iter(train_loader))
resnet = resnet.cuda()
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
outputs = resnet(inputs)
outputs

size of the image
(1600, 1200)
size of the image
(1600, 1200)
size of the image
(1200, 1600)
size of the image
(1024, 768)
size of the image
(1600, 1200)


tensor([[-0.2694,  0.7849,  0.7987,  ...,  1.5062,  1.1582,  1.0888],
        [-3.4425, -1.6673, -1.3675,  ..., -1.4573,  0.9107, -0.5987],
        [-0.3540, -1.7301,  0.4896,  ..., -2.7695,  1.3383, -0.2298],
        [-1.6446,  0.9563, -1.9116,  ..., -0.8069,  0.7711,  2.9739],
        [-0.3990, -0.5433, -0.5574,  ..., -0.1002,  0.6307,  0.8562]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [0]:
/content/gdrive/My Drive/Colab Notebooks/master_research/train_images/7130/8f2da412d32b1873.jpg